In [106]:
import glob
import pandas as pd
import numpy as np
import json

### Merge separate data files into one data frame `(df_main)`

In [107]:
df_master = pd.DataFrame()

files = glob.glob('../data/*.csv')
files

['../data/indeed-8-5.csv',
 '../data/indeed-8-7.csv',
 '../data/linkedin-8-5.csv',
 '../data/linkedin-8-9.csv',
 '../data/linkedin-8-8.csv',
 '../data/indeed-8-8.csv',
 '../data/indeed-8-9.csv']

In [108]:
for i, filepath in enumerate(files):
    df = pd.read_csv(filepath)
    filename_elements = filepath.split('/')[2].split('.')[0].split('-')  # splits up the filename to extract platform & search date info
    df['platform'] = filename_elements[0]  # first element is the platform
    df['search_date'] = f'{filename_elements[1]}-{filename_elements[2]}-2019'  # second/third elements are the month & date of the search
    
    frames = [df_master, df]
    df_master = pd.concat(frames, sort=True)

In [109]:
df_master = df_master.dropna(subset=['description'])  # Drop rows with null values in description
df_master.head(20)


,company,description,employment_type,full_desc,industries,job_function,location,platform,post_date,salary,search_city,search_date,search_role,seniority_level,title,url
0,Noom Inc.,"\n At Noom, we use scientifically proven meth...",NaN,NaN,NaN,NaN,"New York, NY",indeed,30+,NaN,NaN,8-5-2019,Data Scientist,NaN,Data Engineer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Strategic Financial Solutions,\n Overview\nDo you love numbers and finding ...,NaN,NaN,NaN,NaN,"New York, NY 10018",indeed,18,NaN,NaN,8-5-2019,Data Scientist,NaN,Lead Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Averity,\n We are one of the world's premiere travel ...,NaN,NaN,NaN,NaN,"New York, NY",indeed,12,"$175,000 - $225,000 a year",NaN,8-5-2019,Data Scientist,NaN,"Manager, Data Science",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,Lenmar Consulting,\n Medical Data Scientist \n Must have at lea...,NaN,NaN,NaN,NaN,"Woodcliff Lake, NJ",indeed,0,$75 an hour,NaN,8-5-2019,Data Scientist,NaN,Medical Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,Memorial Sloan Kettering,"\n <h2 class=""jobSectionHeader"">Company Overv...",NaN,NaN,NaN,NaN,"New York, NY 10017",indeed,16,NaN,NaN,8-5-2019,Data Scientist,NaN,Sr Strategic Consultant / Data Scientist (Pati...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
5,Digitalogy,\n What you will do\n\nResponsible for assist...,NaN,NaN,NaN,NaN,"New York, NY",indeed,5,$50 - $80 an hour,NaN,8-5-2019,Data Scientist,NaN,Data Scientist,https://www.indeed.com/rc/clk?jk=83e3f665b22e5...
6,Viacom,"\n <h2 class=""jobSectionHeader"">Overview and ...",NaN,NaN,NaN,NaN,"New York, NY 10036",indeed,7,NaN,NaN,8-5-2019,Data Scientist,NaN,Jr. Data Scientist,https://www.indeed.com/rc/clk?jk=1942a17d2bd16...
7,Butterfly Network,\n Job Description<br/>\n<br/>\nWe are lookin...,NaN,NaN,NaN,NaN,"New York, NY",indeed,3,NaN,NaN,8-5-2019,Data Scientist,NaN,Data Scientist,https://www.indeed.com/rc/clk?jk=14bc4e97c18a1...
8,Celonis,\n Are you ready for a new challenge?\n \n <b...,NaN,NaN,NaN,NaN,"New York, NY 10036",indeed,2,NaN,NaN,8-5-2019,Data Scientist,NaN,Data Scientist,https://www.indeed.com/rc/clk?jk=755c879e19b03...
9,Macy's,\n Job Overview:<br/>\n\n\nAs part of the Sup...,NaN,NaN,NaN,NaN,"New York, NY 10001",indeed,14,NaN,NaN,8-5-2019,Data Scientist,NaN,Associate Data Scientist,https://www.indeed.com/rc/clk?jk=126152117b65e...


### Observations
- Inconsistency in which features are included. There are many null values throughout the data
- Very inconsistent/sparse salary column 

### Clean descriptions

### Group job titles into categories

In [111]:
all_job_titles = list(df_main['title'].unique())

# data scientists
data_scientist_count = 0

# engineers
data_engineer_count = 0
machine_learning_engineer_count = 0

# analysts
data_analyst_count = 0
business_analyst_count = 0

# research
ml_ai_research_count = 0
statistics_count = 0

job_title_dictionary = {}

for job_title in all_job_titles:
    if 'data scien' in job_title.lower():
        job_title_dictionary[job_title] = ['Data Scientist']
        data_scientist_count += 1
    if 'data' in job_title.lower() and 'engineer' in job_title.lower():
        data_engineer_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('Data Engineer')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['Data Engineer']
    if ('machine learning' in job_title.lower() or 'ml' in job_title.lower())\
        and ('engineer' in job_title.lower() or 'developer' in job_title.lower() or 'software' in job_title.lower()):
        machine_learning_engineer_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('Machine Learning Engineer')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['Machine Learning Engineer']
    if 'analyst' in job_title.lower() and 'data' in job_title.lower():
        data_analyst_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('Data Analyst')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['Data Analyst']
    if 'analyst' in job_title.lower() and ('business' in job_title.lower() or 'product' in job_title.lower()\
        or 'market' in job_title.lower() or 'strategy' in job_title.lower() or 'management' in job_title.lower()):
        business_analyst_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('Business Analyst')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['Business Analyst']
    if 'research' in job_title.lower() and ('machine learning' in job_title.lower() or 'ml' in job_title.lower()\
        or 'ai' in job_title.lower() or 'artificial intelligence' in job_title.lower()):
        ml_ai_research_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('ML/AI Researcher')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['ML/AI Researcher']
    if 'statistic' in job_title.lower() and ('model' in job_title.lower() or 'research' in job_title.lower()):
        statistics_count += 1
        if job_title in job_title_dictionary.keys():  # if the job title has already been given one of the previous labels, add on to it
            labels = job_title_dictionary[job_title]
            labels.append('Statistical Modeler/Researcher')
            job_title_dictionary[job_title] = labels
        else:
            job_title_dictionary[job_title] = ['Statistical Modeler/Researcher']

with open('job_title_dictionary.json', 'w') as json_file:
    json.dump(job_title_dictionary, json_file)

In [112]:
data_scientist_count

199

In [113]:
data_engineer_count

197

In [114]:
machine_learning_engineer_count

219

In [115]:
data_analyst_count

227

In [116]:
business_analyst_count

363

In [117]:
statistics_count

13

In [118]:
# deprecated code - retaining these keywords for seniority
#if 'junior' in job_title.lower()\
#    or ('jr' in job_title.lower() and 'sr' not in job_title.lower())\
#    or 'intern' in job_title.lower():
#    job_title_dictionary[job_title] = ['Junior Data Scientist']
#    junior_data_scientist_count += 1
#elif 'senior' in job_title.lower()\
#    or 'sr' in job_title.lower()\
#    or 'lead' in job_title.lower()\
#    or 'chief' in job_title.lower()\
#    or 'principal' in job_title.lower()\
#    or 'manager' in job_title.lower():
#    job_title_dictionary[job_title] = ['Senior Data Scientist']
#    senior_data_scientist_count += 1